<a href="https://colab.research.google.com/github/jasperzyzhang/BadmintonNotificationBot/blob/main/Badminton_Email_bot_publish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import smtplib
#outlook account info
from_email = #email account you want to login'jasperzhangbot@outlook.com'
pass_word = #password as a string eg. 'password'
#test = 3 #testing mode
test = 2 #index to real contact info

In [ ]:
def send_message(to_email,subject,body):
  msg = f'Subject: {subject}\n\n{body}'
  try:
      smtpObj = smtplib.SMTP('smtp-mail.outlook.com', 587)
  except Exception as e:
      print(e)
      smtpObj = smtplib.SMTP_SSL('smtp-mail.outlook.com', 465)
  smtpObj.ehlo()
  smtpObj.starttls()
  smtpObj.login(from_email,pass_word) 
  smtpObj.sendmail(from_email, to_email, msg) # Or recipient@outlook
  smtpObj.quit()
  pass

In [ ]:
#send_message("send@outlook.com","test","testsender")

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#read in worksheet
worksheet = gc.open_by_url() # enter the url of the sheet object


# get_all_values gives a list of rows.
june = worksheet.get_worksheet(0).get_all_values()
july = worksheet.get_worksheet(1).get_all_values()
contact = worksheet.get_worksheet(test).get_all_values()
# Convert to a DataFrame and render.
df_june = pd.DataFrame.from_records(june)
df_june = df_june.rename(columns=df_june.iloc[1]).drop([0,1]).reset_index()
df_june.loc[pd.to_datetime(df_june['Date']).notnull()]
#contact info
df_contact = pd.DataFrame.from_records(contact)
df_contact = df_contact.rename(columns=df_contact.iloc[0]).drop(df_contact.index[0]).reset_index()
all_spots = ['Spot1','Spot2','Spot3','Spot4','Spot5','Spot6','Spot7','Spot8','Spot9','Spot10']

In [ ]:
def notify_badminton(date_to_notify):
  row0 = df_june.loc[df_june['Date'] == date_to_notify]
  attendee = np.array(row0[all_spots])
  contact = np.array(df_contact['Preferred ShortCut'])
  indices = np.where(np.in1d(contact, attendee))[0]
  contact_to_notify = df_contact.loc[indices]
  game_date = row0['Date'].to_string(index=False)
  game_time = row0['Time'].to_string(index=False)
  game_dofw = row0['DofW'].to_string(index=False)
  game_timeinfo = game_date + " " + game_dofw + " " + game_time 
  game_address = row0['Address'].to_string(index=False)
  game_organizer = row0['Organizer'].to_string(index=False)
  game_emt = row0['Emt'].to_string(index=False)
  for index, row in contact_to_notify.iterrows():
      name = row['Name']
      email_address = row['Email Address']
      new_subject = "Upcoming Badminton Game on " + game_date
      new_msg = "Hello " + name + ",\n" + game_organizer + " invites you to join the badminton game on " + game_timeinfo + \
      ",\nthis game will be held at " + game_address + \
      " , please come on time with your equipments and send email transfer to " + game_emt + "\n\n"\
        "Have fun!\n\nBadminton Bot created by Jasper Zhang" 
      send_message(email_address,new_subject,new_msg)
      print('sent to ' + name)


In [ ]:
notify_badminton('2022-06-23')

sent to Sichao Zhu
sent to Harris Chen
sent to Yue Zhang
